Repository of all helper functions in the algorithm and what they do as I put together the final file 


Imports:

In [30]:
## Imports and Stop Word 
from nltk.cluster.util import cosine_distance
import bs4 as bs
import urllib.request
import re
import nltk
import nltk.tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import heapq
import math
import numpy as np
import networkx as nx 

## For now
from helpers import *
stop_words = set(stopwords.words('english'))

Pull Text from Wikipedia:

In [27]:
def pull_text_or_link(article_text, wiki_link = False):
    
    ## Read in article
    if wiki_link:
        scraped_data = urllib.request.urlopen(article_text)
        article = scraped_data.read()
        parsed_article = bs.BeautifulSoup(article,'lxml')
        paragraphs = parsed_article.find_all('p')
        article_text = ""
        
        for p in paragraphs:
            article_text += p.text
        
        article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
        article_text = re.sub(r'\s+', ' ', article_text) # Remove extra space
        
        formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text)
        formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    else:
        ## Remove everything else 
        formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text)
        formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

    return formatted_article_text, article_text

In [18]:
test1_file = open('test.txt','r')
test1 = test1_file.read()
test1_file.close()

In [9]:
test2 = 'https://en.wikipedia.org/wiki/India'

In [ ]:
test1

In [20]:
test1 = re.sub(r'\[[0-9]*\]', ' ', test1)
test1 = re.sub(r'\s+', ' ', test1)

In [23]:
fart_test1, art_test1 = pull_text_or_link(test1)

In [28]:
fart_test2, art_test2 = pull_text_or_link(test2, wiki_link=True)

Get word freq, and sentence list

In [31]:
def sentence_tokenize(formatted_article_text, article_text, stop_words = stop_words):
    
    ## Tokenize the sentences in the OG article text, initialize stopwords
    sentence_list = nltk.sent_tokenize(article_text)
    word_frequencies = {} # A dictionary of words and how often they show up 

    ## Fill up word_freq dict with (you guessed it) word frequencies
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stop_words:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    return word_frequencies, sentence_list

In [32]:
wf1, sl1 = sentence_tokenize(fart_test1, art_test1)

In [35]:
wf2, sl2 = sentence_tokenize(fart_test2, art_test2)

In [38]:
def idf_variables(sent_list, word_freq):
    word_idf = {}
    for word in word_freq.keys():
        word_idf[word] = idf(sent_list, word_freq[word])
    sent_broken = [sent_tokenize(sent) for sent in sent_list]
    sent_cleaned = [word_tokenize(str(sent)) for sent in sent_broken]
    
    return word_idf, sent_cleaned

In [39]:
wf_idf1, sc1 = idf_variables(sl1, wf1)

In [41]:
def cosine_similarity(sent1, sent2, word_idf):
        
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    for word in sent1:
        if word in word_idf.keys():
            vector1[all_words.index(word)] += word_idf[word]
    
    for word in sent2:
        if word in word_idf.keys():
            vector2[all_words.index(word)] += word_idf[word]
    
    return 1 - cosine_distance(vector1, vector2)

In [47]:
def score_sentences(sent_cleaned, word_idf):
    similarity_matrix= np.zeros((len(sent_cleaned), len(sent_cleaned)))
    for idx1 in range(len(sent_cleaned)):
        for idx2 in range(len(sent_cleaned)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = cosine_similarity(sent_cleaned[idx1], sent_cleaned[idx2], word_idf)
    similarity_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(similarity_graph)
    return scores
        

In [ ]:
# scores = nx.pagerank(similarity_graph)

In [46]:
wf2_idf2, sc2 = idf_variables(sl2, wf2)

In [48]:
scores2 = score_sentences(sc2, wf2_idf2)

In [56]:
## Sort Top N IDF Values
def top_n(scores, sent_list, num_sents):
    all_idx = list(scores.keys())
    all_sents = [sent_list[idx] for idx in all_idx]
    all_stats = list(scores.values())
    final_sents = []
    top_idx = list(np.argsort(all_stats)[-1*num_sents:])
    top_idx.sort()
    for idx in top_idx:
        final_sents.append(all_sents[idx])
    return final_sents

In [53]:
all_idx2 = list(scores2.keys())

In [54]:
all_sents2 = [sl2[idx] for idx in all_idx2]

In [58]:
final_sents2 = top_n(scores2, sl2, 25)

In [59]:
wf2_idf2, sc2 = idf_variables(sl2, wf2)

["India's population grew from 361 million in 1951 to 1.211 billion in 2011.",
 'The earliest known modern human remains in South Asia date to about 30,000 years ago.',
 'These gradually developed into the Indus Valley Civilisation, the first urban culture in South Asia, which flourished during 2500–1900 BCE in what is now Pakistan and western India.',
 'By the 4th and 5th centuries, the Gupta Empire had created a complex system of administration and taxation in the greater Ganges Plain; this system became a model for later Indian kingdoms.',
 'The Indian early medieval age, from 600 to 1200 CE, is defined by regional kingdoms and cultural diversity.',
 'Embracing a strong Shaivite tradition and building upon the military technology of the sultanate, the empire came to control much of peninsular India, and was to influence South Indian society for long afterwards.',
 'By this time, with its economic power severely curtailed by the British parliament and having effectively been made an 

In [61]:
scores1 = score_sentences(sc1, wf_idf1)

In [66]:
final_sents1 = top_n(scores1, sl1, 25)

In [67]:
final_sents1

['because of the rapid passage of large amounts of indigestible plant material through the short, straight digestive tract.',
 'It is also noted, however, that such rapid passage of digesta limits the potential of microbial digestion in the gastrointestinal tract, limiting alternative forms of digestion.',
 'Given this voluminous diet, the giant panda defecates up to 40 times a day.',
 "The limited energy input imposed on it by its diet has affected the panda's behavior.",
 'It has been estimated that an adult panda absorbs 54.8â€“66.1â€‰mg of cyanide a day through its diet.',
 "Two of the panda's most distinctive features, its large size and round face, are adaptations to its bamboo diet.",
 'Anthropologist Russell Ciochon observed: "[much] like the vegetarian gorilla, the low body surface area to body volume [of the giant panda] is indicative of a lower metabolic rate.',
 'This lower metabolic rate and a more sedentary lifestyle allows the giant panda to subsist on nutrient poor reso